Generate tables of statistics per-label for the supplement

In [ ]:
import sys
import glob
import re
import fnmatch
import math
import datetime
import re
import os
from os import listdir
from os.path import join, isfile, basename

import itertools

import numpy as np
from numpy import float32, int32, uint8, dtype, genfromtxt

from scipy.stats import ttest_ind

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, FormatStrFormatter
%matplotlib inline

sys.path.append("../") # go to parent dir
import template_common as tc

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
# Global options
write_files_and_tables = True

In [ ]:
dist_df, jac_df, hess_df, timeMem_df = tc.readStatTables()
grouped_label_table, total_table = tc.groupTables( dist_df, jac_df, hess_df, timeMem_df, tc.template_list  )

In [ ]:
template_order = ['JFRC2010','JFRC2013','JRC2018','FCWB','Tefor']
alg_order = ['ANTs A','ANTs B','ANTs C','CMTK A','CMTK B','CMTK C', 'Elastix A', 'Elastix B']
pd.options.display.float_format = '{:,.3f}'.format

row_order_tuples = [(x,y) for x,y in itertools.product( template_order, alg_order )]

date_string = datetime.date.today().strftime('%Y%m%d')

for label in tc.labels:

    print( label )
    df_l = dist_table[ dist_table.LABEL == label ]
    df_writeme = df_l[['TEMPLATE','ALG','DISTANCE_mean','DISTANCE_std','DISTANCE_p10','DISTANCE_median','DISTANCE_p90','count']]
    df_writeme.columns = ['Template','Algorithm','Mean','Std dev','10th perc','median','90th perc','N']
    df_writeme = tc.toFloat( df_writeme, ['Mean','Std dev','10th perc','median','90th perc'] )
    df_writeme = df_writeme.set_index( ['Template','Algorithm'])
    
    # Re-order rows as desired
    df_writeme = df_writeme.loc[pd.MultiIndex.from_tuples( row_order_tuples )]
    print( df_writeme )
    
    f = 'label_tables/label_%s_%s.tex' % ( label, date_string )

    if( write_files_and_tables ):
        print( f )
        with open( f, 'w') as tex_file:
            print( '\\begin{table}', file=tex_file )
            print( df_writeme.to_latex( multirow='True'), file=tex_file)
            print( '\caption{{ {} }}'.format( tc.get_label_string( label ).replace('_','\_')), file=tex_file )
            print( '\caption{{ {} }}'.format( tc.get_label_string( label ).replace('_','\_')) )
            print( '\end{table}', file=tex_file )